In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.nn import functional as F

from torch.utils.data import Dataset, DataLoader
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

import pickle as pk
from PIL import Image
import random


PyTorch Version:  1.0.0
Torchvision Version:  0.2.1


## 返回数据库的文件路径数组

In [61]:
file_dir = '../数据集/UCMerced_LandUse/data/test'
def get_img_path(file_dir):
    x = os.listdir(file_dir)
    file_paths = []
    for file_dir_child in x :
        file_dir_child = file_dir+'/'+file_dir_child
        for file in os.listdir(file_dir_child):
            file_paths.append(file_dir_child + '/' + file)
    return file_paths

In [62]:
paths = get_img_path(file_dir)

In [63]:
paths

['../数据集/UCMerced_LandUse/data/test/agricultural/agricultural80.tif',
 '../数据集/UCMerced_LandUse/data/test/agricultural/agricultural81.tif',
 '../数据集/UCMerced_LandUse/data/test/agricultural/agricultural82.tif',
 '../数据集/UCMerced_LandUse/data/test/agricultural/agricultural83.tif',
 '../数据集/UCMerced_LandUse/data/test/agricultural/agricultural84.tif',
 '../数据集/UCMerced_LandUse/data/test/agricultural/agricultural85.tif',
 '../数据集/UCMerced_LandUse/data/test/agricultural/agricultural86.tif',
 '../数据集/UCMerced_LandUse/data/test/agricultural/agricultural87.tif',
 '../数据集/UCMerced_LandUse/data/test/agricultural/agricultural88.tif',
 '../数据集/UCMerced_LandUse/data/test/agricultural/agricultural89.tif',
 '../数据集/UCMerced_LandUse/data/test/agricultural/agricultural90.tif',
 '../数据集/UCMerced_LandUse/data/test/agricultural/agricultural91.tif',
 '../数据集/UCMerced_LandUse/data/test/agricultural/agricultural92.tif',
 '../数据集/UCMerced_LandUse/data/test/agricultural/agricultural93.tif',
 '../数据集/UCMerced_La

## 加载预训练模型

In [49]:
from graduation_pytorch import pooling
PATH = "./my_model/rmac_L=2_UcRemote_resnet34_fine-tune.pt" # L其实=3
def load_net(PATH):
    pretrained_net = models.resnet34(pretrained=True)
    pretrained_net.avgpool = pooling.RMAC() # pooling
    pretrained_net.fc = nn.Sequential()
    pretrained_net.load_state_dict(torch.load(PATH))
    return pretrained_net

In [50]:
net = load_net(PATH)

In [51]:
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

## 加载图库特征  512维

In [3]:
from graduation_pytorch.pooling import rmac

save_path = './features/rmac_L=2_UcRemote21-train-resnet'
from graduation_pytorch.utils import save_data, extract_features

if os.path.exists(save_path):
    fo = open(save_path, 'rb')
    features = pk.load(fo, encoding='bytes')
    fo.close()
    print("Load features.")
else:
    features = extract_features(pretrained_net, Gallery_loader)
    save_data(save_path, features)
    print("Save features to %s" % (save_path))

Load features.


## 提取单个查询特征 => 512 向量

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 提取目标图像的特征  修 
# 这里的input是 Gallery_dataset[k][0] 这种形式  tensor  torch.Size([3, 224, 224])
def query_feature(input, pretrained_net):
    x = input

    x = x.unsqueeze(dim=0)
    x.requires_grad = False

    x.to(device)
    pretrained_net.to(device)
    pretrained_net.eval()
    
    pre = pretrained_net(x)
    pre = pre.data.cpu()
    # norm feature
    prenorm = torch.norm(pre, p=2, dim=1, keepdim=True)
    pre = pre.div(prenorm.expand_as(pre))
    return pre

In [28]:
from torch.utils.data import Dataset, DataLoader
def dataLoader():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    # mean = [0.485, 0.456, 0.406]  std = [0.229, 0.224, 0.225]
    test_augs = transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        normalize
    ])
    Gallery_dataset = datasets.ImageFolder(root='../数据集/UCMerced_LandUse/data/test', transform=test_augs)
#     Gallery_dataset = datasets.ImageFolder(root='../数据集/Google dataset of SIRI-WHU_earth_im_tiff/12class_tif', transform=test_augs)
    return Gallery_dataset
Gallery_dataset = dataLoader()
Gallery_loader = DataLoader(Gallery_dataset,
                           batch_size = 32, shuffle=False)

In [29]:
Gallery_dataset[0][0].shape

torch.Size([3, 224, 224])

In [30]:
pwd

'D:\\360downloads\\毕业论文\\pytorch'

In [45]:
device

device(type='cuda')

In [53]:
## 
query_path = './CBIR/static/image_database/test/airplane/airplane80.tif'
def load_query_image(query_path):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    # mean = [0.485, 0.456, 0.406]  std = [0.229, 0.224, 0.225]
    test_augs = transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        normalize
    ])
    query_image = datasets.folder.default_loader(query_path)
    query_image = test_augs(query_image)
    return query_image
query = load_query_image(query_path)

In [54]:
query.shape

torch.Size([3, 224, 224])

In [55]:
# q_feature = query_feature(query, net)
query = query.unsqueeze(dim=0)
query.to('cpu')
net.to('cpu')
net.eval()
pre = net(query)

In [57]:
pre.shape

torch.Size([1, 512])

In [ ]:
pre